# Notebook for data visualization of the processed Lab Data

### Check WD (change if necessary) and file loading

In [5]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/lab_data" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/lab_data"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis
File Location:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/lab_data
New Working Directory:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/lab_data


## Interactive Raster plot of the lab Data with all the channels

In [6]:
# Interactive Plot for the spiking events
# bokeh docs: https://docs.bokeh.org/en/2.4.1/docs/first_steps/first_steps_1.html
import bokeh.plotting as bplt
from bokeh.io import curdoc
from bokeh.models import BoxAnnotation

# Apply the theme to the plot
curdoc().theme = "light_minimal"  # Can be one of "caliber", "dark_minimal", "light_minimal", "night_sky", "contrast"



### Define the function that creates the raster plot

In [7]:
from math import ceil
from bokeh.models import BasicTickFormatter

"""
create_raster_fig: Create a raster figure with the given parameters

Args:
    title (str): Title of the plot
    x_axis_label (str): Label of the x-axis
    y_axis_label (str): Label of the y-axis
    x (np.ndarray): Array of x values
    y_arrays (list): List of tuples containing the (x, y) pairs for each dot to be plotted
    sizing_mode (str): Sizing mode of the plot
    tools (str): Tools to be added to the plot
    tooltips (str): Tooltips to be added to the plot
    legend_location (str): Location of the legend
    legend_bg_fill_color (str): Background fill color of the legend
    legend_bg_fill_alpha (float): Background fill alpha of the legend
    box_annotation_params (dict): Parameters to create a box annotation
Returns:
    bplt.Figure: The plot
"""
def create_raster_fig(title, x_axis_label, y_axis_label, 
               x, y, dot_size=10,
               sizing_mode=None, tools=None, tooltips=None, box_annotation_params=None):
    # Create the plot
    p = bplt.figure(
        title=title,
        x_axis_label=x_axis_label, 
        y_axis_label=y_axis_label,
        sizing_mode=sizing_mode or "stretch_both",    # Make the plot stretch in both width and height
        tools=tools or "pan, box_zoom, wheel_zoom, hover, undo, redo, zoom_in, zoom_out, reset, save",
        tooltips=tooltips or "Data point @x: @y"
        # y_range=(-0.5, 5)  # Set the range of the y-axis
    )

    # Add the dots to the plot
    p.dot(x, y, size=dot_size, color="blue", alpha=1)

    # Axis settings
    p.xaxis.formatter = BasicTickFormatter(use_scientific=False)

    # Grid settings
    # p.ygrid.grid_line_color = "red"

    # Add a box annotation
    if box_annotation_params is not None:
        inner_box = BoxAnnotation(
            bottom=box_annotation_params["bottom"], 
            top=box_annotation_params["top"], 
            left=box_annotation_params["left"], 
            right= box_annotation_params["right"], 
            fill_alpha=box_annotation_params["fill_alpha"], 
            fill_color=box_annotation_params["fill_color"]
        )
        p.add_layout(inner_box)

    # Return the plot
    return p

## Load the data to raster

In [8]:
import pandas as pd

def read_spike_events(file_path: str):
    """Reads the spike events from the input file and returns them as a numpy array
    TODO: Could try to use a structured array instead?

    Args:
        file_path (str): name of the file containing the spike events
    """
    spike_events = []

    try:
        # Read the spike events from the file
        df = pd.read_csv(file_path, header='infer')
        print(df.head())

        # Detect errors
        if df.empty:
            raise Exception("The input file is empty")

        # Convert the scientific notation values to integers if any exist
        df = df.applymap(lambda x: int(float(x)) if (isinstance(x, str) and 'e' in x) else x)

        # Convert the dataframe to a numpy array. Each row is a spike event. The first column is the time and the second column is the channel idx
        spike_events = df.to_numpy()

        return spike_events
    except Exception as e:
        print("Unable to read the input file: ", file_path, " error:", e)

    return spike_events

In [9]:
# Call the function to read the spike events
file_path = "../lab_data/lab_data_1.csv"
spike_events = read_spike_events(file_path)
print("Spike events: ", spike_events.shape, "\n", spike_events[:10])

    time  channel_idx
0   99.5          229
1  303.6            7
2  502.5          229
3  510.6           71
4  528.0           54
Spike events:  (40020, 2) 
 [[ 99.5 229. ]
 [303.6   7. ]
 [502.5 229. ]
 [510.6  71. ]
 [528.   54. ]
 [540.9   7. ]
 [589.3 225. ]
 [631.6 100. ]
 [633.8 100. ]
 [758.3 229. ]]


## Create the raster plot

In [10]:
# Crete the list of x and y values
x_vals = spike_events[:, 0]
y_vals = spike_events[:, 1]

# Create the raster plot
raster_plot = create_raster_fig("Spike Events", "Time (ms)", "Channel", x_vals, y_vals)

## Show the Plot

In [11]:
# Show the plot
bplt.show(raster_plot)

# Export the plot to a file

In [12]:
file_path = "./lab_data_full_raster.html"

# Customize the output file settings
bplt.output_file(filename=file_path, title="Lab Data Full Raster")

# Save the plot
bplt.save(raster_plot)

'/home/monkin/Desktop/feup/thesis/thesis-lava/src/lab_data/lab_data_full_raster.html'